## Portfolio optimization using Genetic Algorithm

#### Given Data:
Monthly Closing Stock values of ARKK, GLD, GREK, SLV, SMH, TSLA.
Data_1 from March 2017 to March 2020.
Data_2 from March 2015 to March 2018.

### Approach and Tasks:

1. Read the data and combine them into one dataframe.
2. Calculate the historical returns for 3 months, 6 months, 12 months, 24 months and 36 months for each of the stocks.
3. Define **Gene** (Scalar): A fraction of the total capital assigned to a stock.
4. Define **Chromosome** (1D Array): Set of genes i.e. fractions of total capital assigned to each stock.
        Check! Sum of each chromosome should be equal to 1.
5. Generate **Initial Population** (2D Array): A set of randomly generated chromosomes.
6. **Fitness function** (Define a Function):
The **Sharpe ratio**, S, is a measure for quantifying the performance (Fitness) of the portfolio which works on "Maximisation of return (mean) and minimisation of risk (Variance) simultaneously" and is computed as
follows:

                S = (µ − r)/σ

    Here µ is the return of the portfolio over a specified period or Mean portfolio return,
         r is the risk-free rate over the same period and
         σ is the standard deviation of the returns over the specified period or Standard deviation of portfolio return.


    Mean portfolio return = Mean Return * Fractions of Total Capital (Chromosome).
    Risk-free rate = 0.0358 (for US)
    Standard deviation of portfolio return = (chromosome * Standard deviation)**2 + Covariance * Respective weights in chromosome.

7. Select **Elite Population** (Define a Function): It filters the elite chromosomes which have highest returns, which was calculated in fitness function.

8. **Mutation**: A function that will perform mutation in a chromosome. Randomly we shall choose 2 numbers between 0, 5 and those elements we shall swap.

9. Crossover: **Arithmetic Crossover** blends the genes of two parent chromosomes to create a new offspring. The new offspring's genes are determined by taking a weighted average of the genes from each parent chromosome.
The offsprings are created according to the equation:
offspring A = alpha ∗ parent1 + (1 − alpha) ∗ parent2
offspring B = (1 − alpha) ∗ parent1 + alpha ∗ parent2

Where alpha is a random number between 0 and 1.
Input: 2 Parents
Output: 2 Children (1d Array)

10. **Next Generation** (define a Function): A function which does mutation,mating or crossover based on a probability and builds a new generation of chromosomes.

11. **Iterate the process**: Iterate the whole process till there is no change in maximum returns or for fixed number of iterations.

#### Pre-requisite tasks:

In [34]:
import numpy as np
import pandas as pd
from functools import reduce
import yfinance as yf
import os

### Task #1:
#### Read the data and combine them into one dataframe.

In [35]:
folder_path = 'Data_1'
files = os.listdir(folder_path)
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    temp = pd.read_csv(file_path, usecols=['Date', 'Close'])
    temp.columns = ['Date', file.replace('.csv', '')]
    dfs.append(temp)

stocks = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)
print(stocks.shape)
stocks.head()


(37, 7)


,Date,ARKK,GLD,GREK,SLV,SMH,TSLA
0,Mar 2020,44.00,148.05,17.07,13.05,117.14,34.93
1,Feb 2020,52.84,148.38,22.35,15.53,131.97,44.53
2,Jan 2020,51.80,149.33,28.56,16.82,137.57,43.37
3,Dec 2019,50.05,142.90,30.39,16.68,141.41,27.89
4,Nov 2019,50.43,137.86,29.76,15.92,132.91,22.00


### Task #2:
#### Calculate the historical returns for 3 months, 6 months, 12 months, 24 months and 36 months for each of the stock.

**Stock Return**:
The formula for the total stock return is the appreciation in the price divided by the original price of the stock.


In [36]:
def hist_return(months):
    """ It calculates Stock returns for various months and returns a dataframe.
        Input: Months in the form of a list.
        Output: Historical returns in the form of a DataFrame. """
    idx=[]
    df=pd.DataFrame()
    for mon in months:
        temp=(stocks.iloc[0,1:] - stocks.iloc[mon,1:])/(stocks.iloc[mon,1:])
        idx.append(str(mon)+'_mon_return')
        df=pd.concat([df, temp.to_frame().T], ignore_index=True)
    df.index=idx
    return df    

In [37]:
hist_stock_returns=hist_return([3,6,12,24,36])
hist_stock_returns

,ARKK,GLD,GREK,SLV,SMH,TSLA
3_mon_return,-0.120879,0.036039,-0.438302,-0.217626,-0.171629,0.25242
6_mon_return,0.02588,0.066105,-0.390139,-0.180276,-0.016704,1.174969
12_mon_return,-0.058421,0.213425,-0.292289,-0.07969,0.101665,0.871919
24_mon_return,0.126184,0.176962,-0.413402,-0.153147,0.123322,0.968997
36_mon_return,0.836394,0.247052,-0.285176,-0.243478,0.469577,0.883019


### Task #3:
Define **Gene** (Scalar): A fraction of the total capital assigned to a stock. Lets address them as weights.

    Gene can be a fractional value between 0 to 1, such as 0.32 of ARKK or 0.21 of GLD or 0.56 of SLV.

In [38]:
gene = np.random.rand()
gene

0.5125179071611349

In [39]:
import time
def gen_mc_grid(rows, cols, n, N):
        np.random.seed(seed=int(time.time()))  # init random seed
        layouts = np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout
        positionX = np.random.randint(0, cols, size=(N * n * 2))
        positionY = np.random.randint(0, rows, size=(N * n * 2))
        ind_rows = 0  # index of layouts from 0 to n-1
        ind_pos = 0  # index of positionX, positionY from 0 to N*n*2-1
        while ind_rows < n:
            layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
            if np.sum(layouts[ind_rows, :]) == N:
                ind_rows += 1
            ind_pos += 1
            if ind_pos >= N * n * 2:
                print("Not enough positions")
                break
        return layouts

def gen_mc_grid_with_NA_loc(rows, cols, n, N,NA_loc):
        np.random.seed(seed=int(time.time()))  # init random seed
        layouts = np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout, NA loc is 0

        layouts_NA= np.zeros((n, rows * cols), dtype=np.int32)  # one row is a layout, NA loc is 2
        for i in NA_loc:
            layouts_NA[:,i-1]=2

        positionX = np.random.randint(0, cols, size=(N * n * 2))
        positionY = np.random.randint(0, rows, size=(N * n * 2))
        ind_rows = 0  # index of layouts from 0 to n-1
        ind_pos = 0  # index of positionX, positionY from 0 to N*n*2-1
        N_count=0
        while ind_rows < n:
            cur_state=layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]
            if cur_state!=1 and cur_state!=2:
                layouts[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols]=1
                layouts_NA[ind_rows, positionX[ind_pos] + positionY[ind_pos] * cols] = 1
                N_count+=1
                if np.sum(layouts[ind_rows, :]) == N:
                    ind_rows += 1
                    N_count=0
            ind_pos += 1
            if ind_pos >= N * n * 2:
                print("Not enough positions")
                break
        return layouts,layouts_NA

In [40]:
gen_mc_grid(5, 5, 100, 50)
gen_mc_grid_with_NA_loc(5, 5, 100, 50,range(10))

Not enough positions
Not enough positions


(array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[2, 2, 2, ..., 1, 1, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        ...,
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2]]))

### Task #4:
Define **Chromosome** (1D Array): Set of genes i.e. fractions of total capital assigned to each stock. Set of weights.

Its a 1d Array of the fractional values of all the stocks such that sum of the array will not be over 1. 
As we have 6 company stocks, we shall generate 6 fractional values (genes) which constitues 1 chromosome.
    
**Why sum should be equal to 1?** As these are fraction of the total capital, we are assuming total capital to be 1 unit.
    
**How to make sure sum =1?** Just generate 6 random numbers and then calculate a factor which is 1 / [sum of random numbers]. Finally multiply each of the random numbers with that factor. The sum will be 1.

In [41]:
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

In [42]:
child=chromosome(6)
print(child,sum(child))

[0.0782396  0.19110359 0.26171903 0.19830425 0.24227762 0.02835591] 0.9999999999999999


### Task #5:

Generate **Initial Population** (2D Array): A set of randomly generated chromosomes

In [43]:
n=6 # Number of stocks = 6
pop_size=100 # initial population = 100

population = np.array([chromosome(n) for _ in range(pop_size)])
print(population.shape)
print(population)

(100, 6)
[[0.19668449 0.11671003 0.13999779 0.1609354  0.17406766 0.21160463]
 [0.10902459 0.04261961 0.19929717 0.31111973 0.26797192 0.06996697]
 [0.07153115 0.04338975 0.19122828 0.28450421 0.23225375 0.17709285]
 [0.19544694 0.18565261 0.05441118 0.11627118 0.26086597 0.18735212]
 [0.20727957 0.16470538 0.1959122  0.19504776 0.1308809  0.10617419]
 [0.17941512 0.27103599 0.01427795 0.29388871 0.1613452  0.08003703]
 [0.23334331 0.16367918 0.14296062 0.0077071  0.24151394 0.21079585]
 [0.18426651 0.18687984 0.16224619 0.13911102 0.05750965 0.26998677]
 [0.25073143 0.11837816 0.20377416 0.14893218 0.09307934 0.18510473]
 [0.45281991 0.11606682 0.13522724 0.18465603 0.10111093 0.01011908]
 [0.09139568 0.10089226 0.27165849 0.08317188 0.17109181 0.28178989]
 [0.18465822 0.37876377 0.11158245 0.26502953 0.05307746 0.00688856]
 [0.20555683 0.07510685 0.21894645 0.17125464 0.19587499 0.13326024]
 [0.33883945 0.12919194 0.09935656 0.17991561 0.10058734 0.1521091 ]
 [0.19195877 0.30923231 0

### Task #6:

**Fitness function** (Define a Function): 
The Sharpe ratio, S, is a measure for quantifying the performance (Fitness) of the portfolio and is computed as
follows:
                
                S = (µ − r)/σ
    
    Here µ is the return of the portfolio over a specified period or Mean portfolio return, 
         r is the risk-free rate over the same period and 
         σ is the standard deviation of the returns over the specified period or Standard deviation of portfolio return.

      
Mean portfolio return = Mean Return * Fractions of Total Capital (Chromosome).

Risk-free rate = 0.0358 (For US)

Standard deviation of portfolio return = (chromosome * Standard deviation)**2 + Covariance * Respective weights in chromosome.

 #### Fitness function Sub Task 1:
 Calculate Mean, Standard deviation and covariance of the Historical stock returns.

In [44]:
# Convert to numeric columns from Object datatypes.
print(hist_stock_returns.info())
cols=hist_stock_returns.columns
hist_stock_returns[cols] = hist_stock_returns[cols].apply(pd.to_numeric, errors='coerce')
print(hist_stock_returns.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3_mon_return to 36_mon_return
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ARKK    5 non-null      object
 1   GLD     5 non-null      object
 2   GREK    5 non-null      object
 3   SLV     5 non-null      object
 4   SMH     5 non-null      object
 5   TSLA    5 non-null      object
dtypes: object(6)
memory usage: 280.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 3_mon_return to 36_mon_return
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ARKK    5 non-null      float64
 1   GLD     5 non-null      float64
 2   GREK    5 non-null      float64
 3   SLV     5 non-null      float64
 4   SMH     5 non-null      float64
 5   TSLA    5 non-null      float64
dtypes: float64(6)
memory usage: 280.0+ bytes
None


#### Calculate covariance of historical returns

In [45]:
cov_hist_return=hist_stock_returns.cov()

print(cov_hist_return)

# For ease of calculations make covariance of same variable as zero.
for i in range(6):
    cov_hist_return.iloc[i][i]= 0
    
cov_hist_return

          ARKK       GLD      GREK       SLV       SMH      TSLA
ARKK  0.150806  0.023540  0.015924 -0.013799  0.085191  0.034491
GLD   0.023540  0.008543  0.005382  0.001323  0.019340  0.012109
GREK  0.015924  0.005382  0.005000  0.000916  0.012833  0.008554
SLV  -0.013799  0.001323  0.000916  0.004024 -0.003112  0.006550
SMH   0.085191  0.019340  0.012833 -0.003112  0.056132  0.034883
TSLA  0.034491  0.012109  0.008554  0.006550  0.034883  0.119122


,ARKK,GLD,GREK,SLV,SMH,TSLA
ARKK,0.000000,0.023540,0.015924,-0.013799,0.085191,0.034491
GLD,0.023540,0.000000,0.005382,0.001323,0.019340,0.012109
GREK,0.015924,0.005382,0.000000,0.000916,0.012833,0.008554
SLV,-0.013799,0.001323,0.000916,0.000000,-0.003112,0.006550
SMH,0.085191,0.019340,0.012833,-0.003112,0.000000,0.034883
TSLA,0.034491,0.012109,0.008554,0.006550,0.034883,0.000000


#### Calculate the mean of historical returns

In [46]:
mean_hist_return=hist_stock_returns.mean()
mean_hist_return

ARKK    0.161832
GLD     0.147917
GREK   -0.363862
SLV    -0.174844
SMH     0.101246
TSLA    0.830265
dtype: float64

#### Calculate Standard deviation of historical returns:

In [47]:
sd_hist_return=hist_stock_returns.std()
sd_hist_return

ARKK    0.388338
GLD     0.092429
GREK    0.070711
SLV     0.063434
SMH     0.236922
TSLA    0.345141
dtype: float64

#### Fitness function Sub Task 2:
 Calculate Expected portfolio return and portfolio variance.

#### Calculate Expected returns of portfolio.

In [48]:
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))

In [49]:
mean_portfolio_return(population[0])

0.1633263005636655

#### Calculate portfolio variance.

In [50]:
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(6):
        for j in range(6):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

In [51]:
var_portfolio_return(population[0])

0.02860008302964092

#### Risk free factor.

In [52]:
rf= 0.0358

#### Fitness Function of a portfolio.

In [53]:
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child)-rf)/np.sqrt(var_portfolio_return(child))

In [54]:
fitness_fuction(population[7])

1.0399044441726852

### Task #7:
Select **Elite Population** (Define a Function): It filters the elite chromosomes which have highest returns, which were calculated in fitness function.

In [55]:
def Select_elite_population(population, frac=0.3):
    ''' Select elite population from the total population based on fitness function values.
        Input: Population and fraction of population to be considered as elite.
        Output: Elite population.'''
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

In [56]:
print(len(Select_elite_population(population, frac=0.3)))
Select_elite_population(population, frac=0.3)

30


[array([0.18336804, 0.31811473, 0.08886276, 0.04018681, 0.06573994,
        0.30372772]),
 array([0.18494917, 0.02780954, 0.11231092, 0.10948739, 0.14478333,
        0.42065965]),
 array([0.08978931, 0.17550206, 0.19367429, 0.08153532, 0.07717845,
        0.38232058]),
 array([0.26260605, 0.12177488, 0.00215577, 0.04712908, 0.24960796,
        0.31672626]),
 array([0.02745493, 0.16966588, 0.18227709, 0.18518312, 0.07501607,
        0.36040292]),
 array([0.27879326, 0.23206418, 0.00903557, 0.12600544, 0.09599097,
        0.25811057]),
 array([0.18905292, 0.28179539, 0.12909152, 0.03224224, 0.09768938,
        0.27012855]),
 array([0.12584744, 0.24280147, 0.07931443, 0.17360884, 0.12303915,
        0.25538867]),
 array([0.09786486, 0.23386981, 0.10231653, 0.04086606, 0.27859549,
        0.24648725]),
 array([0.11572267, 0.09006493, 0.1066896 , 0.16826615, 0.22363408,
        0.29562257]),
 array([0.10887632, 0.25231553, 0.18034827, 0.00515379, 0.19113459,
        0.2621715 ]),
 array([0.

In [57]:
[fitness_fuction(x) for x in population][:3]

[0.7540777287855851, -0.4242777621572671, 0.25320094575654245]

### Task #8:
**Mutation**: A function that will perform mutation in a chromosome. 
            
    Randomly choose 2 numbers between [0, 5] and those elements should be swapped.


In [58]:
def mutation(parent):
    ''' Randomy choosen elements of a chromosome are swapped
        Input: Parent
        Output: Offspring (1D Array)'''
    child=parent.copy()
    n=np.random.choice(range(6),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(6),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

In [59]:
mutation(population[1]),population[1]

(array([0.31111973, 0.04261961, 0.19929717, 0.10902459, 0.26797192,
        0.06996697]),
 array([0.10902459, 0.04261961, 0.19929717, 0.31111973, 0.26797192,
        0.06996697]))

### Task #9:
Crossover: **Arithmetic Crossover** blends the genes of two parent chromosomes to create a new offspring. The new offspring's genes are determined by taking a weighted average of the genes from each parent chromosome.

In [60]:
def Arithmetic_crossover(parent1,parent2):
    """The offsprings are created according to the equation:
            offspring A = alpha ∗ parent1 + (1 − alpha) ∗ parent2
            offspring B = (1 − alpha) ∗ parent1 + alpha ∗ parent2

                Where alpha is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)"""
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

In [61]:
Arithmetic_crossover(population[2],population[3])

(array([0.16082582, 0.14590546, 0.09263683, 0.1632742 , 0.25287194,
        0.18448576]),
 array([0.10615227, 0.08313691, 0.15300263, 0.23750119, 0.24024779,
        0.17995921]))

In [62]:
for i in population[:30]:
    for j in population[:30]:
        print(Arithmetic_crossover(i,j))

(array([0.19668449, 0.11671003, 0.13999779, 0.1609354 , 0.17406766,
       0.21160463]), array([0.19668449, 0.11671003, 0.13999779, 0.1609354 , 0.17406766,
       0.21160463]))
(array([0.17511892, 0.09848275, 0.15458627, 0.19788285, 0.19716943,
       0.17675978]), array([0.13059015, 0.06084689, 0.18470869, 0.27417228, 0.24487016,
       0.10481182]))
(array([0.12761349, 0.07624523, 0.16827144, 0.22913191, 0.20618004,
       0.19255789]), array([0.14060215, 0.08385455, 0.16295464, 0.2163077 , 0.20014137,
       0.19613959]))
(array([0.19554596, 0.18013595, 0.06125966, 0.11984513, 0.25392053,
       0.18929276]), array([0.19658546, 0.12222669, 0.13314931, 0.15736145, 0.1810131 ,
       0.20966399]))
(array([0.1981781 , 0.12347604, 0.14788016, 0.16574429, 0.16797953,
       0.19674188]), array([0.20578596, 0.15793937, 0.18802983, 0.19023887, 0.13696903,
       0.12103694]))
(array([0.18006814, 0.26520031, 0.01903192, 0.28886122, 0.16182629,
       0.08501212]), array([0.19603146, 0.12254

### Task#10:
**Next Generation**: A function which does mutation,mating or crossover based on a probability and builds a new generation of chromosomes.

In [63]:
def next_generation(pop_size,elite,crossover=Arithmetic_crossover):
    """ Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6.
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array)."""
    new_population=[]
    elite_range=range(len(elite))
#     print(elite_range)
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
#         print(mutate_or_crossover)
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(6):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if sum(range(chk))>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            new_population.extend([c1,c2])
    return new_population

In [64]:
elite=Select_elite_population(population)
next_generation(100,elite,Arithmetic_crossover)[:3]

[array([0.12324711, 0.26724371, 0.26711983, 0.03868823, 0.06668382,
        0.2370173 ]),
 array([0.07098714, 0.24536575, 0.13656842, 0.1750044 , 0.1680345 ,
        0.20403979]),
 array([0.18408168, 0.21387751, 0.14714974, 0.11876032, 0.05920279,
        0.27692797])]

### Task #11:
**Iterate the process**: Iterate the whole process till their is no change in maximum returns/min risk or for fixed number of iterations.

## Backtesting using Arithmetic crossover method

In [65]:
n=6 # Number of stocks = 6
pop_size=100 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 and Expected_risk > 0.0005) or iteration <= 40:
    print('Iteration:',iteration)
    population = next_generation(100,elite,Arithmetic_crossover)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1


print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

Iteration: 0
Expected returns of 0.33906355727831966 with risk of 0.03842016938725171

Iteration: 1
Expected returns of 0.279087829707064 with risk of 0.02608595807350531

Iteration: 2
Expected returns of 0.279087829707064 with risk of 0.02608595807350531

Iteration: 3
Expected returns of 0.28466679470468614 with risk of 0.02775726721101769

Iteration: 4
Expected returns of 0.28576147655699724 with risk of 0.0280506805753743

Iteration: 5
Expected returns of 0.28425850471718167 with risk of 0.02770946375255639

Iteration: 6
Expected returns of 0.29469878290228524 with risk of 0.027649657450577708

Iteration: 7
Expected returns of 0.2924910696315632 with risk of 0.02745440258782199

Iteration: 8
Expected returns of 0.3159306550711009 with risk of 0.032736420180195056

Iteration: 9
Expected returns of 0.3111534445905419 with risk of 0.03174991738700644

Iteration: 10
Expected returns of 0.29120876999233625 with risk of 0.02773557071261876

Iteration: 11
Expected returns of 0.296315573925

[None, None, None, None, None, None]

In [66]:
# Downloading historical monthly closing prices of the assets in portfolio from Yahoo Finance
tickers = ['ARKK', 'GLD', 'GREK', 'SLV', 'SMH', 'TSLA']
start_date = '2020-03-01'
end_date = '2021-03-31'
prices = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

weights = np.array(elite[0])
investment = 10000

# Calculate how much money was invested in each stock
invested = investment * weights

# Calculate the return on investment for each stock
returns = prices.iloc[-1] / prices.iloc[0] - 1
roi = invested * returns

total_roi = sum(roi)
ending_portfolio_value = investment + total_roi
actual_return = (ending_portfolio_value - investment) / investment

print('Portfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':','{:.2%}'.format(elite[0][i])) for i in range(6)]

print('\nExpected returns of {} with risk of {}\n'.format('{:.2%}'.format(Expected_returns), '{:.2%}'.format(Expected_risk)))


print('Total portfolio data at the end date of the investment:\n')
for i in range(len(tickers)):
    print(tickers[i], ':')
    print('Invested -', '${:,.2f}'.format(invested[i]))
    print('Return -', '{:.2%}'.format(returns[i]))
    print('ROI -', '${:,.2f}'.format(roi[i]))
    print('End worth -', '${:,.2f}'.format(invested[i]+(roi[i])), '\n')
print('Total Portfolio Worth:', '${:,.2f}'.format(ending_portfolio_value))
print('Total ROI -', '${:,.2f}'.format(total_roi))
print('Actual Return -', '{:.2%}'.format(actual_return))

[*********************100%***********************]  6 of 6 completed
Portfolio of stocks after all the iterations:

ARKK : 9.45%
GLD : 21.48%
GREK : 4.64%
SLV : 15.28%
SMH : 15.02%
TSLA : 34.12%

Expected returns of 30.20% with risk of 3.01%

Total portfolio data at the end date of the investment:

ARKK :
Invested - $945.04
Return - 114.96%
ROI - $1,086.42
End worth - $2,031.45 

GLD :
Invested - $2,148.49
Return - 5.61%
ROI - $120.53
End worth - $2,269.02 

GREK :
Invested - $464.36
Return - 20.07%
ROI - $93.19
End worth - $557.55 

SLV :
Invested - $1,528.46
Return - 43.06%
ROI - $658.14
End worth - $2,186.60 

SMH :
Invested - $1,501.51
Return - 74.60%
ROI - $1,120.13
End worth - $2,621.65 

TSLA :
Invested - $3,412.14
Return - 327.38%
ROI - $11,170.74
End worth - $14,582.88 

Total Portfolio Worth: $24,249.15
Total ROI - $14,249.15
Actual Return - 142.49%
